In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
os.getcwd()

'/Users/chance/Desktop/LH_home/ksc'

In [3]:
folder = os.listdir()

In [147]:
file = []
for f in folder:
    if ".csv" in f:
        data = pd.read_csv("./" + f, encoding="cp949", skiprows=15)
        file.append(data[list(data.columns[:11])])

gajwa = pd.concat(file)
gajwa = gajwa.dropna(axis=0)

In [149]:
gajwa.head()

,시군구,번지,도로조건,계약면적(㎡),전월세구분,계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명
1,경상남도 진주시 가좌동,1***,12m미만,15.0,월세,202107,4,150,30,2016.0,가좌길48번길
2,경상남도 진주시 가좌동,1***,8m미만,15.0,월세,202108,11,200,32,2015.0,가좌길60번길
4,경상남도 진주시 가좌동,5**,25m미만,15.0,월세,202102,20,300,36,2019.0,가좌길
5,경상남도 진주시 가좌동,5**,25m미만,15.0,월세,202106,21,300,29,2019.0,가좌길
6,경상남도 진주시 가좌동,1***,8m미만,15.0,월세,202108,27,300,30,2018.0,가좌길44번길
